# Introdução

Sistemas de recomendação são ferramentas que filtram e processam informações, retornando um resultado que faça sentido para o contexto em que está sendo aplicado. Esses sistemas utilizam algoritmos específicos para oferecer sugestões personalizadas, facilitando a tomada de decisão do usuário. Neste estudo, buscamos compreender os principais algoritmos empregados em sistemas de recomendação e avaliar seu desempenho aplicando-os ao "Book-Crossing Dataset", um conjunto de dados que reúne informações sobre livros, avaliações e interações de usuários.

A partir dessa análise, pretendemos identificar os métodos mais eficazes para realizar recomendações precisas dentro deste contexto.

# Materiais e Métodos:

## Dependências

As bibliotecas necessárias para rodar os nossos testes de caso.

In [1]:
%pip install kagglehub
%pip install pandas
%pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import kagglehub
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
import os

/usr/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## DataSet

Nós escolhemos o Book-Crossing Dataset, que relaciona usuários, livros e avaliações, permitindo analisar padrões de leitura e preferências. Com isso, aplicaremos algoritmos de recomendação para sugerir livros personalizados, baseados em interações anteriores.

### Importa DataSet

In [ ]:
path = kagglehub.dataset_download("somnambwl/bookcrossing-dataset")

books = pd.read_csv(path+"/Books.csv", sep=";", header=0)
users = pd.read_csv(path+"/Users.csv", sep=";", low_memory=False, header=0)
ratings = pd.read_csv(path+"/Ratings.csv", sep=";", low_memory=False, header=0)

### Visualização dos dados importados

In [4]:
books.head()

,ISBN,Title,Author,Year,Publisher
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton & Company


In [5]:
users.head()

,User-ID,Age
0,1,NaN
1,2,18
2,3,NaN
3,4,17
4,5,NaN


In [6]:
ratings.head()

,User-ID,ISBN,Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


### Junção dos dados em um unico dataset

In [9]:
min_user_ratings = 5  # Usuarios que avaliaram no minimo 5 livros
min_book_ratings = 10  # Livros com no minimo 10 avaliações

user_counts = ratings['User-ID'].value_counts()
book_counts = ratings['ISBN'].value_counts()

filtered_ratings = ratings[
    ratings['User-ID'].isin(user_counts[user_counts >= min_user_ratings].index) &
    ratings['ISBN'].isin(book_counts[book_counts >= min_book_ratings].index)
]

### Criação da matriz de interações

In [11]:
interaction_matrix = filtered_ratings.pivot(index='User-ID', columns='ISBN', values='Rating')
interaction_matrix = interaction_matrix.fillna(0)